In [4]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

### Performance Data (rawdf)

In [5]:
rawdf = pd.read_excel('UNI Agency Performance Dashboard_11-08-2023.xlsm', 
                       sheet_name='Input - Performance Data', dtype=str)

# Dataframe One (Volume Data)

### Required Columns

In [68]:
col1 = ['Collection Tier', 'Servicer','Place YrMo', 'Accounts', 'Balance',
        'Current Volume', 'Current Balance']

# Numeric Columns

numeric1 = col1[3:]

In [69]:
dfOne = rawdf[col1]

### Filters Required Tiers

In [70]:
# List of tier names required

tierList = ['Dismissed Bankruptcy Prime', 'Value Tier A+', 'Value Tier A',
            'Value Tier B', 'Value Tier C', 'Value Tier D', 'Value Tier E', 
            'Value Tier F', 'Value Tier G', 'Spanish Value Tier A',
            'Spanish Value Tier B', 'Spanish Value Tier C', 'Auto Value Tier A',
            'Auto Value Tier B', 'Auto Value Tier C', 'Auto Value Tier D',
            'Auto Value Tier E', 'Sallie Mae 1', 'Sallie Mae 2' ]

In [71]:
dfOne = dfOne[dfOne['Collection Tier'].isin(tierList)]

### Filters Required Months

In [72]:
def RequiredMonths(monthsNum, current=False):

    currDate = date.today()

    # List of dates of each required month

    if current == True: MonthList = [currDate.strftime('%Y%m')]
    else: MonthList = []

    for i in range(monthsNum):

        prevDate = currDate - timedelta(days=currDate.day)
        currDate = prevDate

        # Only the month and year of previous date is required

        MonthList.append(prevDate.strftime('%Y%m'))
    
    return MonthList

In [509]:
monthList6 = RequiredMonths(6)

dfOne = dfOne[dfOne['Place YrMo'].isin(monthList6)]

### Changes dataType of Columns

In [74]:
dfOne[numeric1] = dfOne[numeric1].apply(pd.to_numeric, errors='coerce', downcast='float')

### Function for each column Dictionary

In [415]:
dict1 = {'Accounts': 'sum', 'Balance': 'sum',
        'Current Volume': 'sum', 'Current Balance': 'sum'}

### Dataframe with Multi-Index (Tier, Servicer)

In [11]:
dfVolumn = dfOne.groupby(['Collection Tier', 'Servicer']).agg(dict1)

### Dataframe with Multi-Index Column

In [12]:
dfVolumn.reset_index(inplace=True)

In [13]:
dfPivot = dfVolumn.pivot(index='Collection Tier', columns='Servicer', values=numeric1)

### Final DataFrame (First 7 Columns)

In [14]:
# Columns required

TotalSegAcc = dfPivot.loc[:, ('Accounts', 'Total')].rename('Total Acc')
TotalSegAmt = dfPivot.loc[:, ('Balance', 'Total')].rename('Total Amt')
UniSegAcc = dfPivot.loc[:, ('Accounts', 'UNI')].rename('UNI Acc')
UniSegAmt = dfPivot.loc[:, ('Balance', 'UNI')].rename('UNI Amt')
UniCurrAcc = dfPivot.loc[:, ('Current Volume', 'UNI')].rename('UNI Current Acc')
UniCurrAmt = dfPivot.loc[:, ('Current Balance', 'UNI')].rename('UNI Current Amt')

# Joining Columns to make a final DataFrame

dfOneFinal = pd.concat([TotalSegAcc, TotalSegAmt, UniSegAcc,
                       UniSegAmt, UniCurrAcc, UniCurrAmt], axis=1)

# Sorts dataframe according to tiers required

dfOneFinal = dfOneFinal.reindex(tierList)

# Adds a new column with % Account Column

dfOneFinal['% Acc'] = dfOneFinal['UNI Acc'] / dfOneFinal['Total Acc']

dfOneFinal.to_csv('Check1.csv')

# Dataframe Two (% Liq to Maximum)

### Required Columns

In [132]:
col2 = ['Collection Tier', 'Servicer','Place YrMo', 'Accounts', 'Balance',
        'Current Volume', 'Liq Rate']

# Numeric Columns

numeric2 = col2[3:]

In [133]:
dfTwo = rawdf[col2]

### Filters Required Tiers

In [134]:
dfTwo = dfTwo[dfTwo['Collection Tier'].isin(tierList)]

### Filters Required Months

In [135]:
dfTwo = dfTwo[dfTwo['Place YrMo'].isin(monthList14)]

### Changes dataType of Columns

In [136]:
dfTwo[numeric2] = dfTwo[numeric2].apply(pd.to_numeric, errors='coerce', downcast='float')

In [137]:
dfTwo['SumCurrVol'] = dfTwo.groupby(['Collection Tier', 'Servicer'])['Current Volume'].transform('sum')

### Grouping Data on Collection Tier and YrMo

In [138]:
groupList = []

for tier_YrMo, group in dfTwo.groupby(['Collection Tier', 'Place YrMo']):

    # Sorting grouped dataframe Descendingly

    sorted_group = group.sort_values(by='Liq Rate', ascending=False)

    # Gets Max Servicer name & maximum Liq Rate

    MaxLiqRate = sorted_group.iloc[0, 6]
    MaxLiqRateServicer = sorted_group.iloc[0, 1]

    group['MaxLiqRateServicer'] = MaxLiqRateServicer
    group['MaxLiqRate'] = MaxLiqRate

    groupList.append(group)

# Gets whole dataframe together

dfTwo = pd.concat(groupList)

In [139]:
def DollarDeficit(row):

    if row['SumCurrVol'] == 0: return 0

    elif row['Servicer'] == row['MaxLiqRateServicer']: return 0

    else: 
        
        return (row['Balance'] * row['Liq Rate']) - (row['MaxLiqRate'] * row['Balance'])

In [140]:
dfTwo['$ToLeader'] = dfTwo.apply(lambda x: DollarDeficit(x), axis = 1)

### Filter Required Months

In [141]:
dfTwo = dfTwo[dfTwo['Place YrMo'].isin(monthList1)]

### Filtering UNI

In [142]:
dfUNI = dfTwo[dfTwo['Servicer'] == 'UNI'][['Collection Tier', 'Place YrMo', '$ToLeader']]

In [143]:
dfPivot2 = dfUNI.pivot(index='Collection Tier', columns='Place YrMo', values='$ToLeader')

In [144]:
# Sorts dataframe according to tiers required

dfTwoFinal = dfPivot2.reindex(tierList)

In [145]:
dfTwoFinal.to_csv('Check2.csv')

# Dataframe Three (Ranking)

### Required Columns

In [385]:
col3 = ['Collection Tier', 'Servicer','Place YrMo', 'Accounts', 'Payer Rate']
#        'Current Volume', 'Liq Rate']

# Numeric Columns

numeric3 = col3[3:]

In [386]:
dfThree = rawdf[col3]

### Filters Required Tiers

In [387]:
dfThree = dfThree[dfThree['Collection Tier'].isin(tierList)]

### Filters Servicer

In [388]:
dfThree = dfThree[dfThree['Servicer'] != 'Total']

### Filters Required Months

In [389]:
dfThree = dfThree[dfThree['Place YrMo'].isin(monthList14)]

### Monthly POB

In [390]:
# Servicers in each Tier are being computed for each month
# whoever has the maximum payer rate for that month gets 100% POB

In [391]:
dfThree[numeric3] = dfThree[numeric3].apply(pd.to_numeric, errors='coerce', downcast='float')

In [392]:
dfThree

,Collection Tier,Servicer,Place YrMo,Accounts,Payer Rate
0,Auto Value Tier A,CCS,202210,1653.0,0.007864
1,Auto Value Tier A,CCS,202211,882.0,0.023810
2,Auto Value Tier A,CCS,202212,1270.0,0.006299
3,Auto Value Tier A,CCS,202301,1286.0,0.023328
4,Auto Value Tier A,CCS,202304,709.0,0.009873
...,...,...,...,...,...
1243,Value Tier G,UNI,202308,94787.0,0.000105
1244,Value Tier G,UNI,202309,31657.0,0.000063
1245,Value Tier G,UNI,202310,25110.0,0.000080
1246,Value Tier G,UNI,202311,68.0,0.000000


In [393]:
groupList3 = []

for tier_YrMo, group in dfThree.groupby(['Collection Tier', 'Place YrMo']):

    # Sorting grouped dataframe Descendingly

    sorted_group = group.sort_values(by='Payer Rate', ascending=False)

    # Gets Max Payer Rate

    MaxPayRate = sorted_group.iloc[0, 4]

    group['MonthlyPOB'] = group['Payer Rate'] / MaxPayRate

    group['MonthlyPOB'] = group['MonthlyPOB'].replace([np.inf, -np.inf, np.nan], 0)

    groupList3.append(group)

# Gets whole dataframe together

dfThree = pd.concat(groupList3)

### Servicer Avg POB (Last 14 months)

In [394]:
# For Last 14 months mean of POB is calculated, If a servicer 
# is has payer rate for last 14 months consecutively
# that servicer will get 100% Average. (Ideal case)

In [395]:
dfThree = dfThree.groupby(['Collection Tier', 'Servicer']).agg({'MonthlyPOB':'mean'}).reset_index()

In [396]:
dfThree.rename(columns = {'MonthlyPOB':'ServiceAvgPOB'}, inplace=True)

### Payer Rate %

In [397]:
groupList3 = []

for tier, group in dfThree.groupby(['Collection Tier']):

    MaxAvgPOB = group['ServiceAvgPOB'].max()

    group['Payer Rate %'] = group['ServiceAvgPOB'] / MaxAvgPOB

    groupList3.append(group)

# Gets whole dataframe together

dfThree = pd.concat(groupList3)

In [398]:
dfThree = dfThree[dfThree['Servicer'] == 'UNI']

In [399]:
dfThree = dfThree.drop(columns=['Servicer', 'ServiceAvgPOB'])

In [400]:
# Sorts dataframe according to tiers required

dfThree.set_index('Collection Tier', inplace=True)

In [401]:
dfThreeFinal = dfThree.reindex(tierList)

In [402]:
dfThreeFinal

,Payer Rate %
Collection Tier,
Dismissed Bankruptcy Prime,1.000000
Value Tier A+,0.924843
Value Tier A,0.899578
Value Tier B,0.807935
Value Tier C,0.634452
Value Tier D,0.568010
Value Tier E,0.515401
Value Tier F,0.565051
Value Tier G,0.333062


# Dataframe Four (CLIQ To Leader)

In [268]:
col4 = ['Collection Tier', 'Servicer','Place YrMo', 'Balance', 'Liq Rate']

# Numeric Columns

numeric4 = col4[3:]

In [270]:
dfFour = rawdf[col4]

### Filters Required Months

In [271]:
dfFour = dfFour[dfFour['Place YrMo'].isin(monthList6)]

### Filters Servicer

In [272]:
dfFour = dfFour[dfFour['Servicer'] != 'Total']

In [273]:
dfFour[numeric4] = dfFour[numeric4].apply(pd.to_numeric, errors='coerce', downcast='float')

In [274]:
dfFour['Liq Amount'] = dfFour['Balance'] * dfFour['Liq Rate']

In [275]:
dfFour = dfFour.groupby(['Collection Tier', 'Servicer']).agg({'Balance':'sum', 'Liq Amount':'sum'}).reset_index()

In [276]:
dfFour['Cliq'] = dfFour['Liq Amount'] / dfFour['Balance']

In [292]:
groupList4 = []

for tier, group in dfFour.groupby(['Collection Tier']):

    MaxCliq = group['Cliq'].max()

    SecondMaxCliq = group['Cliq'][group['Cliq'] < MaxCliq].max()

    group['Cliq To Leader'] = group['Cliq'] / MaxCliq

    group['Rank'] = group['Cliq'].rank(ascending=False).astype('int')

    group['2nd Max Cliq to Leader'] = group['Rank'].apply(lambda x: SecondMaxCliq / MaxCliq if x == 1 else None) 

    group['Total Servicer'] = group['Rank'].max()

    group['Rank (/)'] = group['Rank'].astype('str') + '/' + group['Total Servicer'].astype('str')

    groupList4.append(group)

# Gets whole dataframe together

dfFour = pd.concat(groupList4)

In [293]:
dfFour

,Collection Tier,Servicer,Balance,Liq Amount,Cliq,Cliq To Leader,2nd Max Cliq to Leader,Rank,Total Servicer,Rank (/)
0,Auto Value Tier A,CCS,6.555289e+07,83066.969783,0.001267,0.679524,NaN,2,5,2/5
1,Auto Value Tier A,FBS,8.790899e+07,104976.647092,0.001194,0.640365,NaN,4,5,4/5
2,Auto Value Tier A,JEF,9.352060e+07,111857.181909,0.001196,0.641394,NaN,3,5,3/5
3,Auto Value Tier A,SCI,1.010358e+08,117316.347826,0.001161,0.622661,NaN,5,5,5/5
4,Auto Value Tier A,UNI,1.569399e+08,292661.067554,0.001865,1.000000,0.679524,1,5,1/5
...,...,...,...,...,...,...,...,...,...,...
88,Value Tiers C & D,UNI,4.934703e+08,716413.871957,0.001452,0.914497,NaN,2,2,2/2
89,Value Tiers E & F,JEF,7.678649e+08,383407.839173,0.000499,1.000000,0.892949,1,4,1/4
90,Value Tiers E & F,MAB,1.537217e+08,34131.570952,0.000222,0.444677,NaN,3,4,3/4
91,Value Tiers E & F,SCI,1.559853e+08,30080.469432,0.000193,0.386211,NaN,4,4,4/4


In [294]:
dfFour = dfFour[dfFour['Servicer'] == 'UNI']

In [295]:
dfFour.set_index('Collection Tier', inplace=True)

In [296]:
dfFourFinal = dfFour[['Rank (/)', 'Cliq To Leader', '2nd Max Cliq to Leader']].reindex(tierList)

In [297]:
dfFourFinal

,Rank (/),Cliq To Leader,2nd Max Cliq to Leader
Collection Tier,,,
Dismissed Bankruptcy Prime,1/1,1.000000,NaN
Value Tier A+,2/3,0.996091,NaN
Value Tier A,2/3,0.989747,NaN
Value Tier B,2/3,0.976021,NaN
Value Tier C,2/2,0.913710,NaN
Value Tier D,2/2,0.915055,NaN
Value Tier E,2/4,0.844474,NaN
Value Tier F,2/4,0.988993,NaN
Value Tier G,3/3,0.467989,NaN


# Dataframe Five (Inc. To Leader)

In [612]:
liqCols = [f'Liq Rate {i}' for i in range(14)]

In [613]:
otherCols = ['Collection Tier', 'Servicer','Place YrMo', 'Accounts', 'Balance']

col5 =  otherCols + liqCols

# Numeric Columns

numeric5 = col5[4:]

In [614]:
dfFive = rawdf[col5]

In [615]:
dfFive[numeric5] = dfFive[numeric5].apply(pd.to_numeric, errors='coerce', downcast='float')

C:\Users\MAQ\AppData\Local\Temp\ipykernel_16636\4218597205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFive[numeric5] = dfFive[numeric5].apply(pd.to_numeric, errors='coerce', downcast='float')


### Filters Required Months

In [616]:
dfFive6Months = dfFive[dfFive['Place YrMo'].isin(monthList6)]

### 6 Month Accounts sum of Total

In [617]:
dfFive6Months['Accounts'] = dfFive6Months['Accounts'].astype(int)

C:\Users\MAQ\AppData\Local\Temp\ipykernel_16636\1399360091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFive6Months['Accounts'] = dfFive6Months['Accounts'].astype(int)


In [618]:
dfAccountSum = dfFive6Months.groupby(['Collection Tier', 'Servicer']).agg({'Accounts':'sum'})

In [619]:
# Dictionary for 6 Month Account sum

SixMonthDict = {}

In [620]:
for index, value in dfAccountSum.iterrows():

    SixMonthDict[index] = value.iloc[0]

### Creates Month Offset Column

In [621]:
monthList14 = sorted(monthList14)

In [622]:
offsetDict = {}

monthCount = 13

for month in monthList14:

    offsetDict[month] = monthCount
    monthCount -= 1

### Grouping Data on Collection Tier and Servicer

In [623]:
def StepLiq(row):

    offset = row['Offset']

    return row[f'Liq Rate {offset}']

In [624]:
groupList = []

for index, group in dfFive.groupby(['Collection Tier', 'Servicer']):

    # Adds Offset Column against each month

    group['Offset'] = group['Place YrMo'].apply(lambda x: offsetDict[x])

    # Creates a step Liq Column

    group['Step Liq'] = group.apply(StepLiq, axis=1)

    # Creates Incremental liq Amount

    group['Inc. Liq Amount'] = group['Balance'] * group['Step Liq']

    # Gets group of Last six months

    groupSixMonth = group[group['Place YrMo'].isin(monthList6)]

    # Checks if sum of Accounts of total in last 6 months is 0
    # Then whole group (14 months) is needed to calculate 'Incremental'

    if SixMonthDict[(index[0], 'Total')] == 0:

        groupList.append(group[otherCols + ['Inc. Liq Amount']])

    # If not, only six months of group are needed

    else:

        groupList.append(groupSixMonth[otherCols + ['Inc. Liq Amount']])

# Gets whole dataframe together

dfFive = pd.concat(groupList)

In [625]:
aggFunctions = {'Balance':'sum', 'Inc. Liq Amount':'sum'}

dfFive = dfFive.groupby(['Collection Tier', 'Servicer']).agg(aggFunctions)

In [626]:
dfFive['Incremental'] = dfFive['Inc. Liq Amount'] / dfFive['Balance']

In [627]:
dfFive.reset_index(inplace=True)

In [628]:
dfFive

,Collection Tier,Servicer,Balance,Inc. Liq Amount,Incremental
0,Auto Value Tier A,CCS,6.555289e+07,2741.000022,0.000042
1,Auto Value Tier A,FBS,8.790899e+07,2834.689977,0.000032
2,Auto Value Tier A,JEF,9.352060e+07,3377.930067,0.000036
3,Auto Value Tier A,SCI,1.010358e+08,4936.279957,0.000049
4,Auto Value Tier A,Total,5.049581e+08,23780.630027,0.000047
...,...,...,...,...,...
115,Value Tiers E & F,JEF,7.678649e+08,17255.440077,0.000022
116,Value Tiers E & F,MAB,1.537217e+08,1203.379993,0.000008
117,Value Tiers E & F,SCI,1.559853e+08,1218.819960,0.000008
118,Value Tiers E & F,Total,1.253386e+09,26033.239610,0.000021


In [631]:
groupList = []

for index, group in  dfFive.groupby(['Collection Tier']):

    # Gets Maximum Incremental of group

    MaxInc = group['Incremental'].max()

    group['Incremental To leader'] = group['Incremental'] / MaxInc

    groupList.append(group['Collection Tier', 'Servicer', 'Incremental To leader'])

dfFive = pd.concat(groupList)

In [638]:
dfFive = dfFive[dfFive['Servicer'] == 'UNI'].set_index('Collection Tier')

In [639]:
dfFive.reindex(tierList)

,Servicer,Balance,Inc. Liq Amount,Incremental,Incremental To leader
Collection Tier,,,,,
Dismissed Bankruptcy Prime,UNI,1.688168e+07,2317.059981,1.372529e-04,1.000000
Value Tier A+,UNI,6.895982e+07,18627.119876,2.701155e-04,1.000000
Value Tier A,UNI,2.241297e+08,43191.150068,1.927060e-04,0.735387
Value Tier B,UNI,2.630371e+08,44260.819004,1.682684e-04,0.956747
Value Tier C,UNI,2.334370e+08,11917.109974,5.105066e-05,0.577814
Value Tier D,UNI,2.600333e+08,20990.820258,8.072357e-05,1.000000
Value Tier E,UNI,9.588180e+07,3498.820007,3.649097e-05,1.000000
Value Tier F,UNI,7.993258e+07,2856.779941,3.573987e-05,1.000000
Value Tier G,UNI,2.664063e+08,73.879998,2.773208e-07,0.034521


# Merge DataFrames

In [362]:
dfFinal = pd.merge(dfOneFinal, dfTwoFinal, left_index=True, right_index=True)

In [363]:
dfFinal

,Total Acc,Total Amt,UNI Acc,UNI Amt,UNI Current Acc,UNI Current Amt,% Acc,202305,202306,202307,202308,202309,202310
Collection Tier,,,,,,,,,,,,,
Dismissed Bankruptcy Prime,13881.0,1.688168e+07,13881.0,1.688168e+07,8784.0,1.009879e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Value Tier A+,53685.0,1.921246e+08,19232.0,6.895982e+07,14417.0,5.091335e+07,0.358238,0.000000,-6074.112765,-20695.599293,-4153.790746,0.000000,-5375.160779
Value Tier A,268154.0,6.281761e+08,96507.0,2.241297e+08,68047.0,1.611194e+08,0.359894,-10125.669302,-16865.249084,-63381.120098,0.000000,0.000000,-2047.841034
Value Tier B,440432.0,7.396212e+08,158552.0,2.630371e+08,112825.0,1.917376e+08,0.359992,-64025.911749,-25091.123534,-5105.794829,0.000000,0.000000,-10128.646552
Value Tier C,656690.0,9.188850e+08,174597.0,2.334370e+08,130844.0,1.782163e+08,0.265874,-34135.337082,-26804.713542,-6931.267818,0.000000,0.000000,-15151.230398
Value Tier D,879077.0,1.025241e+09,232489.0,2.600333e+08,190575.0,2.147584e+08,0.264469,-25521.361960,-8524.505775,-9730.616399,0.000000,0.000000,-8125.901588
Value Tier E,658488.0,6.921891e+08,90626.0,9.588180e+07,81403.0,8.763434e+07,0.137627,-6076.880593,-366.867146,0.000000,-2642.502955,0.000000,0.000000
Value Tier F,569810.0,5.611971e+08,79717.0,7.993258e+07,73498.0,7.455523e+07,0.139901,-2278.356417,-1976.325006,0.000000,0.000000,0.000000,0.000000
Value Tier G,1598743.0,1.424129e+09,296050.0,2.664063e+08,267022.0,2.431544e+08,0.185177,-4229.669312,-2233.472387,-3711.509893,-7990.874823,-1215.548695,-504.961622
